## Importing libraries

In [ ]:
import pandas as pd 
import cbsodata
import json
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

## Loading the data

In [ ]:
# Load the data from the JSON file
with open('data/table_selection.json', 'r', encoding='utf-8') as json_file:
    table_data = json.load(json_file)

# Create a dictionary to store the identifiers and their frequencies
identifier_freq_dict = {}

# Loop through the data and extract the identifier and frequency for each entry
for entry in table_data:
    identifier = entry.get('Identifier', 'N/A')
    frequency = entry.get('Frequency', 'N/A')
    
    # Add the identifier and frequency to the dictionary
    identifier_freq_dict[identifier] = frequency

# Print the dictionary of identifiers and their frequencies
print("Dictionary of Identifiers and Frequencies:")
for identifier, frequency in identifier_freq_dict.items():
    print(f"Identifier: {identifier}, Frequency: {frequency}")

In [ ]:
def fetch_and_save_tables(identifiers, output_folder='data'):
    """
    Fetches data for each table identifier using `cbsodata.get_data()` and saves it to a CSV file,
    skipping files that already exist.
    
    Parameters:
        identifiers (list): List of table identifiers to fetch data for.
        output_folder (str): The folder where the CSV files will be saved (default is 'data').
    
    Returns:
        None
    """
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for identifier in identifiers:
        # Define the output CSV file path
        output_file = f'{output_folder}/{identifier}.csv'
        
        # Check if the file already exists
        if os.path.exists(output_file):
            print(f"Skipping {output_file} because it already exists.")
            continue
        
        # Fetch data for the current table identifier
        print(f"Fetching data for table identifier: {identifier}")
        data = pd.DataFrame(cbsodata.get_data(identifier))
        
        # Save the DataFrame to CSV
        data.to_csv(output_file, index=False)
        
        # Print success message
        print(f"Data for {identifier} saved to {output_file}")

# Example usage
fetch_and_save_tables(identifier_freq_dict.keys())

In [ ]:
# Modified function to load CSV files and immediately convert monthly data to quarterly
def load_csv_files(file_paths, identifier_dict):
    """
    Load multiple CSV files into DataFrames, prefix column names with the file name, 
    and convert monthly tables to quarterly.
    
    Parameters:
        file_paths (list): List of file paths to CSV files.
        identifier_dict (dict): Dictionary of identifiers and their frequencies.
    
    Returns:
        list: List of DataFrames with prefixed column names.
    """
    dataframes = []
    
    for file_path in file_paths:
        file_name = file_path.split('/')[-1].split('.')[0]
        frequency = identifier_dict.get(file_name, 'Kwartaal')  # Default to 'Kwartaal' if not specified

        # Load CSV file
        df = pd.read_csv(file_path)
        
        # Prefix column names with filename
        df = df.add_prefix(f"{file_name}_")
        
        # Rename specific columns to prevent prefixing
        for col in ['Perioden', 'BedrijfstakkenBranchesSBI2008', 'ID', 'Jaar', 'Kwartaal']:
            if f"{file_name}_{col}" in df.columns:
                df = df.rename(columns={f"{file_name}_{col}": col})
                
        # If the frequency is monthly, convert it to quarterly
        if frequency == 'Maandelijks':
            print(f"Converting monthly data to quarterly for file: {file_name}")
            df = monthly_to_quarterly(df)

        # Append to list
        dataframes.append(df)
    
    return dataframes

# Function to process the 'Perioden' column
def process_period_column(df, perioden_column='Perioden', year_column='Year', quarter_column='Quarter'):
    """
    Process the 'Perioden' column to extract the latest year and quarter.
    
    Parameters:
        df (DataFrame): DataFrame with 'Perioden' column.
        perioden_column (str): Name of the period column.
        year_column (str): Name of the column for extracted year.
        quarter_column (str): Name of the column for extracted quarter.
    
    Returns:
        DataFrame: DataFrame with new 'Year' and 'Quarter' columns.
    """
    # Extract latest year and quarter from 'Perioden' column
    df[[year_column, quarter_column]] = df[perioden_column].apply(lambda x: pd.Series(extract_latest_year_and_quarter(x)))
    return df

# Helper function to extract latest year and quarter
def extract_latest_year_and_quarter(period_text):
    """
    Extracts the latest year and quarter from a period that may contain a date range (e.g., "2023 april - 2024 maart").
    
    Parameters:
        period_text (str): The period string to process.
    
    Returns:
        tuple: A tuple containing the latest year and quarter.
    """
    # Regular expression to capture year and month/quarter references
    match = re.findall(r'(\d{4})\s*(\w+)', period_text)
    
    if match:
        # Take the last match in the range (latest part of the period)
        latest_year, latest_period_part = match[-1]
        latest_year = int(latest_year)
        
        # Determine the corresponding quarter for the latest period part (month or quarter)
        latest_quarter = extract_quarter(latest_period_part)
        
        return latest_year, latest_quarter
    else:
        # Return None if no valid year/quarter is found
        return None, None

# Helper function to determine the quarter based on months or quarter phrases like '3e kwartaal'
def extract_quarter(period_part):
    """
    Extracts the quarter from the period part (e.g., based on months or quarter keywords).
    
    Parameters:
        period_part (str): The part of the period string after the year.
    
    Returns:
        int: The corresponding quarter (1 to 4), or None if no valid quarter is found.
    """
    # Handle month-based quarters
    if any(month in period_part for month in ['januari', 'februari', 'maart']):
        return 1
    elif any(month in period_part for month in ['april', 'mei', 'juni']):
        return 2
    elif any(month in period_part for month in ['juli', 'augustus', 'september']):
        return 3
    elif any(month in period_part for month in ['oktober', 'november', 'december']):
        return 4
    
    # Handle phrases like '1e kwartaal', '2e kwartaal', etc.
    if '1e' in period_part:
        return 1
    elif '2e' in period_part:
        return 2
    elif '3e' in period_part:
        return 3
    elif '4e' in period_part:
        return 4
    
    # Return None if no valid quarter is found
    return None

def monthly_to_quarterly(df, period_column='Perioden', value_columns=None):
    """
    Convert monthly data to quarterly by summing up the data for each quarter.
    
    Parameters:
        df (DataFrame): The DataFrame with monthly data.
        period_column (str): Name of the column containing period information.
        value_columns (list): List of columns to sum (numeric columns).
    
    Returns:
        DataFrame: DataFrame with quarterly data.
    """
    if value_columns is None:
        value_columns = df.select_dtypes(include=[float, int]).columns.tolist()
    
    # Extract year and month from 'Perioden'
    df['Year'] = df[period_column].apply(lambda x: int(re.search(r'(\d{4})', x).group()))
    df['Month'] = df[period_column].apply(lambda x: extract_month(x))
    
    # Create a 'Quarter' column based on the month
    df['Quarter'] = df['Month'].apply(lambda x: (x - 1) // 3 + 1)
    
    # Group by 'Year' and 'Quarter' and sum the value columns
    quarterly_df = df.groupby(['Year', 'Quarter'])[value_columns].sum().reset_index()
    
    return quarterly_df

# Helper function to extract the month number from a period string
def extract_month(period_text):
    """
    Extracts the month number from the period string.
    
    Parameters:
        period_text (str): The period string to process (e.g., "2023 januari").
    
    Returns:
        int: The month number (1 for January, 2 for February, etc.).
    """
    months = {
        'januari': 1, 'februari': 2, 'maart': 3,
        'april': 4, 'mei': 5, 'juni': 6,
        'juli': 7, 'augustus': 8, 'september': 9,
        'oktober': 10, 'november': 11, 'december': 12
    }
    
    for month_name, month_num in months.items():
        if month_name in period_text.lower():
            return month_num
    
    return None  # Return None if no valid month is found

def rename_sbi_column(df, primary_sbi_column='BedrijfstakkenBranchesSBI2008', backup_sbi_column='BedrijfskenmerkenSBI2008'):
    """
    Renames the 'BedrijfskenmerkenSBI2008' column to 'BedrijfstakkenBranchesSBI2008' if the primary column does not exist.
    
    Parameters:
        df (DataFrame): The DataFrame to process.
        primary_sbi_column (str): The desired primary SBI column name. Default is 'BedrijfstakkenBranchesSBI2008'.
        backup_sbi_column (str): The backup SBI column to rename. Default is 'BedrijfskenmerkenSBI2008'.
    
    Returns:
        DataFrame: Updated DataFrame with the renamed column.
    """
    # Check if the primary column does not exist and the backup column does exist
    if primary_sbi_column not in df.columns and backup_sbi_column in df.columns:
        # Rename 'BedrijfskenmerkenSBI2008' to 'BedrijfstakkenBranchesSBI2008'
        df = df.rename(columns={backup_sbi_column: primary_sbi_column})
        print(f"Renamed '{backup_sbi_column}' to '{primary_sbi_column}'")
    
    return df

# Function to filter by industry
def filter_by_industry(df, industry_column='BedrijfstakkenBranchesSBI2008', valid_industries=None):
    """
    Filter the DataFrame by specific industries.
    
    Parameters:
        df (DataFrame): The DataFrame to filter.
        industry_column (str): Name of the industry column.
        valid_industries (list): List of valid industries to keep.
    
    Returns:
        DataFrame: Filtered DataFrame.
    """
    if valid_industries is None:
        valid_industries = [
            "Q Gezondheids- en welzijnszorg", 
            "G Handel", 
            "C Industrie", 
            "M Specialistische zakelijke diensten", 
            "N Verhuur en overige zakelijke diensten", 
            "O Openbaar bestuur en overheidsdiensten"
        ]
    return df[df[industry_column].isin(valid_industries)]

# Function to filter by year range
def filter_by_year(df, year_column='Year', start_year=2008, end_year=2022):
    """
    Filter the DataFrame by a range of years.
    
    Parameters:
        df (DataFrame): The DataFrame to filter.
        year_column (str): Name of the year column.
        start_year (int): The start year (inclusive).
        end_year (int): The end year (inclusive).
    
    Returns:
        DataFrame: Filtered DataFrame.
    """
    return df[(df[year_column] >= start_year) & (df[year_column] <= end_year)]

def clean_dataframe(df, id_column='ID', year_column='Year', quarter_column='Quarter', sbi_column='BedrijfstakkenBranchesSBI2008', sick_leave_column='80072ned_Ziekteverzuimpercentage_1', columns_to_drop=None):
    """
    Cleans the DataFrame by reordering columns to have 'Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008', and '80072ned_Ziekteverzuimpercentage_1' as the first four columns,
    dropping specified columns, and removing duplicate rows.
    
    Parameters:
        df (DataFrame): The DataFrame to clean.
        year_column (str): The year column name. Default is 'Year'.
        quarter_column (str): The quarter column name. Default is 'Quarter'.
        sbi_column (str): The column name for SBI. Default is 'BedrijfstakkenBranchesSBI2008'.
        sick_leave_column (str): The sick leave column name. Default is '80072ned_Ziekteverzuimpercentage_1'.
        columns_to_drop (list): List of column names to drop. Default is ['ID', 'Perioden'].
    
    Returns:
        DataFrame: Cleaned DataFrame with reordered columns, specified columns dropped, and duplicates removed.
    """
    if columns_to_drop is None:
        columns_to_drop = ['ID', 'Perioden', 'Jaar', 'Kwartaal']

    # Drop the specified columns
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Remove duplicate rows
    df = df.drop_duplicates()

    return df

def group_and_sum(df, group_by_columns):
    """
    Group the DataFrame by specified columns and sum the numeric values.
    
    Parameters:
        df (DataFrame): The DataFrame to group.
        group_by_columns (list): List of columns to group by (e.g., ['Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008']).
    
    Returns:
        DataFrame: Grouped and summed DataFrame.
    """
    # Group by the specified columns and sum the numeric columns
    df = df.groupby(group_by_columns).sum(numeric_only=True).reset_index()
    
    return df

def reorder_columns(df, year_column='Year', quarter_column='Quarter', 
                    sbi_column='BedrijfstakkenBranchesSBI2008', sick_leave_column='80072ned_Ziekteverzuimpercentage_1'):
    """
    Reorders the DataFrame columns to place 'Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008', 
    and '80072ned_Ziekteverzuimpercentage_1' as the first four columns, keeping remaining columns in their original order.
    
    Parameters:
        df (DataFrame): The DataFrame to reorder.
        year_column (str): The year column name.
        quarter_column (str): The quarter column name.
        sbi_column (str): The SBI column name.
        sick_leave_column (str): The sick leave column name.
    
    Returns:
        DataFrame: DataFrame with reordered columns.
    """
    # Define the preferred order of the first four columns
    preferred_columns = [year_column, quarter_column, sbi_column, sick_leave_column]

    # Collect remaining columns in their original order (excluding the preferred ones)
    remaining_columns = [col for col in df.columns if col not in preferred_columns]

    # Combine the preferred columns with the remaining ones
    ordered_columns = preferred_columns + remaining_columns

    # Reorder DataFrame
    return df[ordered_columns]


# Modify the process_tables function to handle monthly data conversion
def process_tables(tables, identifier_dict):
    """
    Process and combine multiple tables into a single DataFrame by:
    - Processing the 'Perioden' column to extract 'Year' and 'Quarter'
    - Filtering by specific industries and year range
    - Reordering columns to have 'Year' and 'Quarter' after 'ID'
    - Dropping unnecessary columns
    - Removing duplicates
    - Grouping by 'Year', 'Quarter', and 'BedrijfstakkenBranchesSBI2008' and summing the data
    
    Parameters:
        tables (list): List of DataFrames.
        identifier_dict (dict): Dictionary of identifiers and their frequencies.
    
    Returns:
        DataFrame: The final combined DataFrame.
    """
    processed_tables = []
    
    for i, table in enumerate(tables):
        identifier = list(identifier_dict.keys())[i]
        frequency = identifier_dict[identifier]

        # If the frequency is monthly, convert the data to quarterly
        if frequency == 'Maandelijks':
            print(f"Converting monthly data to quarterly for identifier: {identifier}")
            table = monthly_to_quarterly(table)

        # Step 1: Process the 'Perioden' column
        period_df = process_period_column(table)
        
        # Step 2: Fill missing 'BedrijfstakkenBranchesSBI2008' with 'BedrijfskenmerkenSBI2008'
        sbi_df = rename_sbi_column(period_df)
        
        # Step 3: Filter by industry
        industry_df = filter_by_industry(sbi_df)
        
        # Step 4: Filter by year range (2008 to 2022)
        year_df = filter_by_year(industry_df)
        
        # Step 5: Clean the DataFrame: reorder columns, drop unnecessary ones, and remove duplicates
        clean_df = clean_dataframe(year_df)
        
        processed_tables.append(clean_df)
    
    # Concatenate all processed tables
    concat_df = pd.concat(processed_tables, ignore_index=True)
    
    # Step 6: Group by 'Year', 'Quarter', and 'BedrijfstakkenBranchesSBI2008' and sum the numeric columns
    grouped_df = group_and_sum(concat_df, ['Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008'])
    
    final_df = reorder_columns(grouped_df)
    
    return final_df

# Load identifiers and frequencies from the JSON file
with open('data/table_selection.json', 'r', encoding='utf-8') as json_file:
    table_data = json.load(json_file)
    identifier_dict = {entry['Identifier']: entry['Frequency'] for entry in table_data}

# Generate file paths based on identifiers
file_paths = [f'data/{identifier}.csv' for identifier in identifier_dict.keys()]

# Load and process data, passing in the identifier dictionary for frequency mapping
tables = load_csv_files(file_paths, identifier_dict)  # Corrected to include `identifier_dict`
final_df = process_tables(tables, identifier_dict)     # Also pass `identifier_dict` here

# Save the final DataFrame to a CSV file
final_df.to_csv('data/merged_tables.csv', index=False)

## Testing

In [ ]:
# Load one specific table file path for testing
file_path = 'data/81588NED.csv'  # Replace 'your_table.csv' with the actual file name
file_name = file_path.split('/')[-1].split('.')[0]

# Load the single table
df = pd.read_csv(file_path)
print("Initial DataFrame:")
print(df.head())

# Step 1: Apply prefixing and special column renaming
df = df.add_prefix(f"{file_name}_")
df = df.rename(columns={
    f"{file_name}_Perioden": 'Perioden',
    f"{file_name}_BedrijfstakkenBranchesSBI2008": 'BedrijfstakkenBranchesSBI2008',
    f"{file_name}_ID": 'ID',
    f"{file_name}_Jaar": 'Jaar',
    f"{file_name}_Kwartaal": 'Kwartaal'
})
print("\nDataFrame after load_csv_files equivalent processing:")
print(df.head())

# Step 2: Process 'Perioden' to extract 'Year' and 'Quarter'
df = process_period_column(df)
print("\nDataFrame after processing 'Perioden' column:")
print(df[['Perioden', 'Year', 'Quarter']].head())

# Step 3: If monthly data, convert to quarterly (based on a mock frequency value for testing)
frequency = 'Maandelijks'  # Replace with the actual frequency from your identifier dictionary if needed
if frequency == 'Maandelijks':
    df = monthly_to_quarterly(df)
    print("\nDataFrame after monthly_to_quarterly conversion:")
    print(df.head())

# Step 4: Rename SBI column if needed
df = rename_sbi_column(df)
print("\nDataFrame after renaming SBI column:")
print(df.head())

# Step 5: Filter by industry
df = filter_by_industry(df)
print("\nDataFrame after filtering by industry:")
print(df.head())

# Step 6: Filter by year range
df = filter_by_year(df, start_year=2008, end_year=2022)
print("\nDataFrame after filtering by year range:")
print(df.head())

# Step 7: Clean the DataFrame
df = clean_dataframe(df)
print("\nDataFrame after cleaning:")
print(df.head())

# Step 8: Reorder columns
df = reorder_columns(df)
print("\nDataFrame after reordering columns:")
print(df.head())

# Step 9: Group and sum the data
df = group_and_sum(df, ['Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008'])
print("\nFinal DataFrame after grouping and summing:")
print(df.head())
